# Model 3

In [2]:
import datetime
import pandas as pd
import numpy as np

In [10]:
rec = pd.read_csv("sp500_financials_with_rec.csv")
rec = rec[['Symbol', 'Consensus']]
rec

,Symbol,Consensus
0,MMM,Buy
1,AOS,Buy
2,ABT,Buy
3,ABBV,Buy
4,ACN,Buy
...,...,...
498,XYL,Buy
499,YUM,Hold
500,ZBRA,Buy
501,ZBH,Buy


In [12]:
import time
import finnhub

# Initialize Finnhub client
finnhub_client = finnhub.Client(api_key="cvdbkj9r01qm9khjrjsgcvdbkj9r01qm9khjrjt0")

# Assuming your existing DataFrame is called df
# Create a list to store all the metrics dictionaries
df = rec
metrics_list = []

# Set delay between requests (in seconds) to avoid rate limiting
DELAY = 1  # Adjust based on your API tier's rate limits

# Iterate through each symbol in the DataFrame
for index, row in df.iterrows():
    symbol = row['Symbol']
    try:
        # Get financial metrics
        metrics = finnhub_client.company_basic_financials(symbol, 'all')
        data = metrics.get('metric', {})
        
        # Add symbol to the data dictionary
        data['Symbol'] = symbol
        
        # Append to metrics list
        metrics_list.append(data)
        
    except Exception as e:
        print(f"Error retrieving data for {symbol}: {str(e)}")
        # Add empty entry with symbol
        metrics_list.append({'Symbol': symbol})
    
    # Add delay between requests
    time.sleep(DELAY)

# Create metrics DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Merge with original DataFrame on Symbol
df = df.merge(metrics_df, on='Symbol', how='left')


In [22]:
df.head()

,Symbol,Consensus,10DayAverageTradingVolume,13WeekPriceReturnDaily,26WeekPriceReturnDaily,3MonthADReturnStd,3MonthAverageTradingVolume,52WeekHigh,52WeekHighDate,52WeekLow,...,roeTTM,roi5Y,roiAnnual,roiTTM,tangibleBookValuePerShareAnnual,tangibleBookValuePerShareQuarterly,tbvCagr5Y,totalDebt/totalEquityAnnual,totalDebt/totalEquityQuarterly,yearToDatePriceReturnDaily
0,MMM,Buy,1.39711,16.2981,16.0120,23.895920,1.18338,156.28,2025-03-03,85.6141,...,96.68,9.06,24.64,21.40,4.8789,4.8789,-6.50,3.4073,3.3951,16.9107
1,AOS,Buy,0.45095,-7.1625,-14.0061,21.616457,0.41758,92.39,2024-07-18,64.5900,...,28.11,21.36,25.69,26.14,10.8260,10.8260,3.30,0.1026,0.1026,-0.8063
2,ABT,Buy,4.54478,10.6536,7.8473,23.027653,1.99508,141.01,2025-03-04,99.7100,...,32.37,13.45,21.69,23.93,23.6860,23.6860,23.87,0.2963,0.2963,11.6612
3,ABBV,Buy,1.42140,22.6415,7.2736,21.688585,2.55190,218.60,2025-03-10,153.5800,...,70.88,8.99,6.07,5.57,0.1439,0.1439,NaN,20.1937,20.1937,20.3433
4,ACN,Buy,1.92213,-10.9306,-6.7961,26.247640,0.89372,398.35,2025-02-05,278.6900,...,26.95,28.50,24.78,25.15,40.5772,42.2970,13.35,0.0362,0.1766,-7.8797


In [32]:
na_counts = df.isna().sum()
pd.set_option('display.max_rows', None)
print("Missing values per column:")
print(na_counts)


Missing values per column:
Symbol                                  0
Consensus                               1
10DayAverageTradingVolume               0
13WeekPriceReturnDaily                  0
26WeekPriceReturnDaily                  1
3MonthADReturnStd                       0
3MonthAverageTradingVolume              0
52WeekHigh                              0
52WeekHighDate                          0
52WeekLow                               0
52WeekLowDate                           0
52WeekPriceReturnDaily                  5
5DayPriceReturnDaily                    0
assetTurnoverAnnual                    18
assetTurnoverTTM                       18
beta                                    0
bookValuePerShareAnnual                 1
bookValuePerShareQuarterly              1
bookValueShareGrowth5Y                 20
capexCagr5Y                            21
cashFlowPerShareAnnual                  0
cashFlowPerShareQuarterly               0
cashFlowPerShareTTM                    17
cashPer

In [38]:
df.to_csv("SP500_financials_with_ratings.csv")

In [70]:
df = pd.read_csv("SP500_financials_with_ratings.csv")
df = df.drop(columns=['Unnamed: 0'])
df['Consensus'].isna().sum()

1

In [78]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Prepare features and target
X = df.drop(columns=['Symbol', 'Consensus', '52WeekHighDate', '52WeekLowDate'])  # Features
y = df['Consensus'].fillna('Sell') # Target

# Encode target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data FIRST to prevent data leakage
X_train, X_test, y_train, y_test = train_test_split(
    X,  # Not yet imputed!
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded  # Maintain class balance
)

# Create pipeline with preprocessing and model
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Uses train-set statistics only
    ('classifier', XGBClassifier(
        objective='multi:softprob',
        num_class=len(label_encoder.classes_),
        eval_metric='mlogloss',
        use_label_encoder=False,
        random_state=42,
        enable_categorical=False  # Ensure numerical data
    ))
])

# Train model
pipeline.fit(X_train, y_train)

# Evaluate
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)  # For probability scores

# Decode labels
y_test_decoded = label_encoder.inverse_transform(y_test)
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Metrics
print(f"Accuracy: {accuracy_score(y_test_decoded, y_pred_decoded):.2f}")
print("\nClassification Report:")
print(classification_report(y_test_decoded, y_pred_decoded))

# Feature importance
feature_importances = pipeline.named_steps['classifier'].feature_importances_
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values('Importance', ascending=False)

print("\nTop 10 Feature Importances:")
print(importance_df.head(10))

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [13:37:01] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1738880277541/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.82

Classification Report:
              precision    recall  f1-score   support

         Buy       0.83      0.98      0.90        81
        Hold       0.67      0.21      0.32        19
        Sell       0.00      0.00      0.00         1

    accuracy                           0.82       101
   macro avg       0.50      0.40      0.41       101
weighted avg       0.79      0.82      0.78       101


Top 10 Feature Importances:
                        Feature  Importance
111                      roaTTM    0.063812
13   bookValuePerShareQuarterly    0.042042
85           pretaxMarginAnnual    0.037902
73               pcfShareAnnual    0.029742
83                 pfcfShareTTM    0.028810
12      bookValuePerShareAnnual    0.023732
25        currentRatioQuarterly    0.020929
1        13WeekPriceReturnDaily    0.020516
23    currentEv/freeCashFlowTTM    0.019921
40                  epsGrowth3Y    0.017961


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [80]:
import joblib

# Save the pipeline (including the model and imputer)
joblib.dump(pipeline, 'model_pipeline.joblib')

# Save the label encoder
joblib.dump(label_encoder, 'label_encoder.joblib')

# Save feature importance
importance_df.to_csv('feature_importance.csv', index=False)

print("\nModel, label encoder, and feature importance exported successfully!")


Model, label encoder, and feature importance exported successfully!
